<a href="https://colab.research.google.com/github/dashitoishi23/PATE-Analysis-on-DL-Models/blob/master/PATE_Analysis_on_Deep_Learning_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PATE Analysis on Deep Learning Models

PATE analysis is carried out on a system of deep learning in which many models after training themselves, are used to label unlabelled data. The unlabelled data provided by all the trained models (we'll call them teachers) are evaluated using differential privacy algorithmic frameworks like PATE to test for leakage of private data (using a term called epsilon).

In [0]:
import torch
import numpy as np
from torch import nn

Download the training and test data (MNIST dataset) using PyTorch like so..

In [2]:
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.MNIST(root="./data", download=True, train=True, transform=transform)

# Download and load the test data
testset = datasets.MNIST(root="./data", download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=1)

0it [00:00, ?it/s]

9920512it [00:06, 1592974.89it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 130577.25it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2138720.98it/s]                            
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 48942.99it/s]            


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


Splitting the training set into 10 more training sets

In [23]:
from torch.utils.data.sampler import SubsetRandomSampler

def split_train_set():
    train_len = len(trainset)
    indices = list(range(train_len))
    split = int(np.floor(0.01 * train_len))
    indice = []  #true indices to be fed to the PATE analysis algorithm
    for image,label in testloader:
      label = np.array(label)
      indice.append(label)
    indice = np.array(indice)
    indice = indice.flatten()
    print(indice.shape)
    trainloaders=[]
    for i in range(100):
        train_idx = indices[(i)*split:(i+1)*split] #splitting training set into 10 equal parts, each a seperate teacher
        train_sampler = SubsetRandomSampler(train_idx)
        trainloader = torch.utils.data.DataLoader(trainset,batch_size=16,sampler=train_sampler)
        trainloaders.append(trainloader)
    
    return trainloaders,indice #return teacher models and the true indices

teachers,indice = split_train_set()
print(len(teachers[0]))
num_teachers = 100
num_examples = 10000  #10,000 unlabelled datapoints in the testset
num_labels = 10

(10000,)
38


Now the interesting part, training the model ;)

In [25]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #using the CUDA cores of Nvidia's Tesla K80
from torch import optim
model = nn.Sequential(nn.Dropout(p=0.2), #setting a dropout with a probability of 0.2 to counter overfitting
                     nn.Linear(784,512), #since the images are 28x28, we are giving a flattened input of 784 pixels
                     nn.ReLU(),
                     nn.Linear(512,256),
                     nn.ReLU(),
                     nn.Linear(256,10),
                     nn.LogSoftmax(dim=1)) #using a Log-Softmax activation for the output layer

criterion = nn.CrossEntropyLoss()  #calculating the cross entropy loss function
optimizer = optim.Adam(model.parameters(),lr=0.001) #using an Adam optimizer with a learning rate of 0.001

#Initialise 2 lists to later properly arrange the predictions made by all the teacher models
preds=[[]*10000]*100
temp_preds = []



model.to(device) #move the model to the GPU

for i in range(num_teachers):
    run_loss,val_loss_gr = [],[]
    epochs = 10  #train each teacher model for 10 epochs
    print(f"Training for teacher:{i+1}")
    for e in range(epochs):
        running_loss = 0
        val_loss = 0
        for images, labels in teachers[i]:
            images = images.view(images.shape[0],-1)
            images, labels = images.to(device),labels.to(device)  #leveraging the computational power of Tesla K80
            output = model(images)
            optimizer.zero_grad()  #setting gradients to 0 before every epoch
            loss = criterion(output,labels)
            loss.backward() #our favourite back propagation
            optimizer.step()
            
            running_loss+=loss.item()
            
        else:
            with torch.no_grad(): #switch off gradient calculation for predictions
                model.eval() #disable dropout while making predictions
                temp = []  #list for storing predictions of all data points
                for image,label in testloader:
                    image = image.view(image.shape[0],-1)
                    image,label = image.to(device),label.to(device)
                    output = model(image)
                    loss = criterion(output,label)
                    ps = torch.exp(output) #finding the probablity distribution for an image
                    top_p,top_class = ps.topk(1,dim=1) #find the class that the model predicted
                    equals = top_class == label.view(*top_class.shape)
                    top_class = top_class.cpu()  #converting a GPU tensor to a normal CPU tensor for further processing
                    temp.append(top_class.flatten())
                    val_loss+=loss.item()
            print(f'Accuracy: {torch.sum(equals.type(torch.FloatTensor))}')
            print(f"Loss:{running_loss/len(teachers[i])} epoch:{e+1}")
            print(f"Val_Loss:{val_loss/len(testloader)} epoch:{e+1}")
        model.train() #re-enable dropout


    temp = np.array(temp)
    print(temp.shape)
    preds[i-1] = temp
    

preds = np.array(preds)
print(preds.shape)#predictions made by all 10 teacher sets stored in this list



            


Training for teacher:1
Accuracy: 1.0
Loss:1.829653893646441 epoch:1
Val_Loss:1.0940711700496264 epoch:1
Accuracy: 1.0
Loss:0.774614238425305 epoch:2
Val_Loss:0.7187759810632793 epoch:2
Accuracy: 1.0
Loss:0.5193068189056296 epoch:3
Val_Loss:0.6176166751860263 epoch:3
Accuracy: 1.0
Loss:0.38384774719413955 epoch:4
Val_Loss:0.622083000412239 epoch:4
Accuracy: 1.0
Loss:0.3666481052182223 epoch:5
Val_Loss:0.5447153028516665 epoch:5
Accuracy: 1.0
Loss:0.29233955886018903 epoch:6
Val_Loss:0.5584666258494656 epoch:6
Accuracy: 1.0
Loss:0.23749362571949237 epoch:7
Val_Loss:0.5413412044947716 epoch:7
Accuracy: 1.0
Loss:0.2173117544305952 epoch:8
Val_Loss:0.6026986540196596 epoch:8
Accuracy: 1.0
Loss:0.1782135854622251 epoch:9
Val_Loss:0.5639615775854875 epoch:9
Accuracy: 1.0
Loss:0.22635175625940687 epoch:10
Val_Loss:0.549513765401478 epoch:10
(10000,)
Training for teacher:2
Accuracy: 1.0
Loss:0.7880634649803764 epoch:1
Val_Loss:0.4644243383131634 epoch:1
Accuracy: 1.0
Loss:0.44100067078282956 ep

Now that we have both the predictions from all the 10 teacher models and also the actual labels, we'll now do the PATE analysis on 2 scenarios

1. With noise in the actual labels and applying PATE analysis

In [32]:
#!pip install syft  #installing the PySyft framework which has the PATE analysis API
from syft.frameworks.torch.differential_privacy import pate

pred = preds.transpose()
new_labels = list()

for an_image in pred:
  label_counts = np.bincount(an_image,minlength=10)
  epsilon = 0.1
  beta = 1/epsilon
  for i in range(len(label_counts)):
    label_counts[i]+= np.random.laplace(0,beta,1)  #adding Laplacian noise to every true indice to hide sensitive information
  
  new_label = np.argmax(label_counts)
  new_labels.append(new_label)
  
new_labels = np.array(new_labels)


  
print(new_labels.shape)
print(pred.shape)


data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=pred.transpose(), indices=new_labels, noise_eps=0.1, delta=1e-5) #calculating the epsilon scores using PATE Analysis
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

(10000,)
(10000, 100)
Data Independent Epsilon: 411.5129254649703
Data Dependent Epsilon: 38.5322839982337


2. Without any noise

We will be adding Laplacian noise to the predicted labels.

In [33]:


data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=pred.transpose(), indices=indice, noise_eps=0.1, delta=1e-5) #calculating the epsilon scores using PATE Analysis
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 411.5129254649703
Data Dependent Epsilon: 38.41832256970914


As we see, there is no difference in epsilon scores even after adding **randomized** Laplacian noise to the predicted labels.

This implies that the differential privacy analysis vis PATE on the dataset was done as per Cynthia Dwork's algorithmic definition of differential privacy